### Instructions:

- You can attempt any number of questions and in any order.  
  See the assignment page for a description of the hurdle requirement for this assessment.
- You may submit your practical for autograding as many times as you like to check on progress, however you will save time by checking and testing your own code before submitting.
- Develop and check your answers in the spaces provided.
- **Replace** the code `raise NotImplementedError()` with your solution to the question.
- Do **NOT** remove any variables other provided markings already provided in the answer spaces.
- Do **NOT** make any changes to this notebook outside of the spaces indicated.  
  (If you do this, the submission system might not accept your work)

### Submitting:

1. Before you turn this problem in, make sure everything runs as expected by resetting this notebook.    
   (You can do this from the menubar above by selecting `Kernel`&#8594;`Restart Kernel and Run All Cells...`)
1. Don't forget to save your notebook after this step.
1. Submit your .ipynb file to Gradescope via file upload or GitHub repository.
1. You can submit as many times as needed.
1. You **must** give your submitted file the **identical** filename to that which you downloaded without changing **any** aspects - spaces, underscores, capitalisation etc. If your operating system has changed the filename because you downloaded the file twice or more you **must** also fix this.  



---

# <mark style="background: #843fa1; color: #ffffff;" >&nbsp;G&nbsp;</mark>&ensp;Topic 12: Graphs & Trees

## Shortest Path First

This practical assumes that you are familiar with the object-oriented sample implementation of graphs presented in the course. This reference code is provided to allow you to extend the provided classes using (single or multi-level) inheritance. Do **not** modify this base code as it is used by the autograder but rather extend the classes in order to answer the questions below. You are not compelled to make use of the provided code and may answer the questions as you see fit.

In [1]:
import heapq
import math

# --- Vertex Class ---
class Vertex:
    def __init__(self, vertex_id):
        self.id = vertex_id
        self.adjacent = {}  # neighbor_id -> weight
        self.distance = math.inf
        self.previous = None

    def add_neighbor(self, neighbor, weight):
        self.adjacent[neighbor] = weight

    def get_id(self):
        return self.id

    def get_previous(self):
        return self.previous

    def get_distance(self):
        return self.distance

# --- Graph Class with Dijkstra ---
class DijkstraGraph:
    def __init__(self):
        self.vertices = {}

    def add_vertex(self, vertex_id):
        if vertex_id not in self.vertices:
            self.vertices[vertex_id] = Vertex(vertex_id)
        return self.vertices[vertex_id]

    def add_edge(self, frm, to, cost):
        self.add_vertex(frm)
        self.add_vertex(to)
        self.vertices[frm].add_neighbor(to, cost)

    def get_vertex(self, vertex_id):
        return self.vertices.get(vertex_id)

    def dijkstra_spf(self, start_vertex):
        # Reset all distances and paths
        for v in self.vertices.values():
            v.distance = math.inf
            v.previous = None

        start_vertex.distance = 0
        heap = [(0, start_vertex.get_id())]  # min-heap of (distance, vertex_id)

        while heap:
            current_distance, current_id = heapq.heappop(heap)
            current_vertex = self.vertices[current_id]

            # Skip outdated entries in heap
            if current_distance > current_vertex.get_distance():
                continue

            for neighbor_id, weight in current_vertex.adjacent.items():
                neighbor = self.vertices[neighbor_id]
                new_dist = current_distance + weight

                if new_dist < neighbor.get_distance():
                    neighbor.distance = new_dist
                    neighbor.previous = current_vertex
                    heapq.heappush(heap, (new_dist, neighbor_id))

# --- Shortest Path Retrieval Function ---
def shortest_path(edges, vstart, vfinish):
    graph = DijkstraGraph()

    # Build the graph
    for frm, to, cost in edges:
        graph.add_edge(frm, to, cost)

    start_vertex = graph.get_vertex(vstart)
    target_vertex = graph.get_vertex(vfinish)

    # Run Dijkstra’s algorithm
    graph.dijkstra_spf(start_vertex)

    # Reconstruct path
    path = []
    current = target_vertex

    while current:
        path.insert(0, current.get_id())
        current = current.get_previous()

    total_weight = target_vertex.get_distance()

    if len(path) < 2 or total_weight == math.inf:
        return [], math.inf  # No valid path found

    return path, total_weight


#### Question 01 &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;(10 Points)

Given an undirected, edge-weighted graph:</br>
<img src="https://zschub.github.io/img/edge_weighted_graph.png" alt="edge-weighted graph" style="align: left"></br>
write a function defined as:
```python
def undirected_shortest_path (edges, vstart, vfinish):
    ...
``` 
that:
* accepts a list of `edges` in the form of tuples `(vertex1, vertex2, weight)`,
* accepts `vstart` and `vfinish` as the IDs of start and stop vertices,
* returns a tuple `(path, weight)` that provides a list of vertex id's along the shortest path from start to finish and the summed weight of edges along the path using Dijkstra's Shortest Path First algorithm, and
* where no path exists, returns an empty list and an infinite distance.

For example, assuming the graph above, the call:
```python
path, distance = directed_shortest_path (edges, '4', '3')
```
would return:
```python
(['4', '0', '2', '3'], 0.81)
```

In [ ]:
class DirectedDijkstraGraph(DijkstraGraph):
    def add_edge(self, frm, to, cost=0):
        """
        Q: What does this method do?
        A: Adds a directed edge from vertex `frm` to vertex `to` with the given `cost`.
        """

        # Q: Ensure both vertices exist in the graph
        if frm not in self.vertices:
            self.add_vertex(frm)
        if to not in self.vertices:
            self.add_vertex(to)

        # Q: Add a directed edge (frm → to) with the given cost
        # A: Only one direction is stored in a directed graph
        self.vertices[frm].add_neighbor(to, cost)


In [ ]:
import math

def directed_shortest_path(edges, vstart, vfinish):
    """
    Q: What does this function do?
    A: Builds a directed, weighted graph using Dijkstra's algorithm,
       finds the shortest path from vstart to vfinish, and returns:
       - path: a list of vertex IDs from start to finish
       - total_weight: the sum of weights along that path
    """

    graph = DirectedDijkstraGraph()

    # Step 1: Add edges to the graph
    for frm, to, cost in edges:
        graph.add_edge(frm, to, cost)

    # Step 2: Get the start and finish vertex objects
    start_vertex = graph.get_vertex(vstart)
    target_vertex = graph.get_vertex(vfinish)

    # Step 3: Handle case where either vertex doesn't exist
    if start_vertex is None or target_vertex is None:
        return [], math.inf

    # Step 4: Run Dijkstra from the starting vertex
    graph.dijkstra_spf(start_vertex)

    # Step 5: Reconstruct the path by following .get_previous() pointers
    path = []
    current = target_vertex
    while current:
        path.insert(0, current.get_id())
        current = current.get_previous()

    # Step 6: Get the total weight from start to finish
    total_weight = target_vertex.get_distance()

    # Step 7: If no real path exists (e.g., disconnected), return empty
    if len(path) < 2 or total_weight == math.inf:
        return [], math.inf

    return path, total_weight


In [4]:
class WeightedDirectedGraph(DirectedDijkstraGraph):
    def add_edge(self, frm, to, cost=0):
        super().add_edge(frm, to, cost)
        super().add_edge(to, frm, cost)

In [6]:
class TreeVertex(Vertex):
    def __init__(self, node):
        super().__init__(node)
        self._parent = None

    def set_parent(self, parent):
        self._parent = parent

    def get_parent(self):
        return self._parent

class TreeGraph(DijkstraGraph):
    def add_edge(self, frm, to, cost=0):
        if frm not in self._vertices:
            self.add_vertex(frm)
        if to not in self._vertices:
            self.add_vertex(to)

        self._vertices[frm].add_neighbour(self._vertices[to], cost)
        self._vertices[to].set_parent(self._vertices[frm])

#### Question 02 &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;(15 Points)

Given a directed, edge-weighted graph:</br>
<img src="https://zschub.github.io/img/directed-edge-weighted.png" alt="edge-weighted directed graph" style="height: 307px; width: 503px; align: left"></br>
write a function defined as:
```python
def directed_shortest_path (dedges, vstart, vfinish):
    ...
``` 
that:
* accepts a list of directed edges in the form of tuples `(vertex1, vertex2, weight)`,
* accepts `vstart` and `vfinish` as the IDs of start and stop vertices, and
* returns a tuple `(path, weight)` that provides a list of vertex id's along the directed path from start to finish and the summed weight of edges along the path using Dijkstra's Shortest Path First algorithm.

For example, the call:
```python
path, distance = directed_shortest_path (directed_edges, 'G', 'C')
```
would return:
```python
(['G', 'D', 'A', 'B', 'C'], 7)
```
If no path exists, the function must return an empty list and a distance (edge-weight) of `math.inf`.

In [7]:
def tree_path(graph, vstart, vfinish):
    path = []
    current = graph.get_vertex(vfinish)
    while current:
        path.insert(0, current.get_id())
        if current.get_id() == vstart:
            break
        current = current.get_parent()
    else:
        return [], math.inf

    return path, len(path) - 1

In [ ]:
class BinaryTreeGraph(TreeGraph):
    def add_edge(self, frm, to, cost=0, side='left'):
        """
        Q: What does this method do?
        A: Adds a directional edge (parent → child) to a binary tree graph,
           connecting `frm` to `to` on either the left or right side.

        Parameters:
        - frm: parent node ID
        - to: child node ID
        - cost: (not used here but maintained for signature compatibility)
        - side: either 'left' or 'right', indicating child side
        """

        # Create frm vertex if it doesn't exist
        if frm not in self._vertices:
            self.add_vertex(frm)

        # Create to vertex if it doesn't exist
        if to not in self._vertices:
            self.add_vertex(to)

        # Set left or right child based on the `side` parameter
        if side == 'left':
            self._vertices[frm].left = self._vertices[to]
        else:
            self._vertices[frm].right = self._vertices[to]

        # Set the parent pointer in the child vertex
        self._vertices[to].set_parent(self._vertices[frm])


In [9]:
# Test cases based on the list directed_edges

p, d = directed_shortest_path (directed_edges, 'G', 'C')
assert p == ['G', 'D', 'A', 'B', 'C'] and d == 7

p, d = directed_shortest_path (directed_edges, 'F', 'E')
assert p == [] and d == math.inf


NameError: name 'directed_edges' is not defined

In [ ]:
# Testing Cell (Do NOT modify this cell)

## Trees

As for graphs, this reference code is provided to allow you to extend the classes using (single or multi-level) inheritance. Do **not** modify this base code as it is used by the autograder but rather extend the classes in order to answer the questions below. You are not compelled to make use of the provided code and may answer the questions as you see fit.

In [ ]:
# Do NOT modify - please extend or ignore.

class TreeVertex:

    def __init__ (self, _value = None):
        # User domain payload of the TreeVertex
        self._value = _value
        
        # Left and right sided children
        self._left = None
        self._right = None
        
    def get_value (self):
        return self._value
    
    def set_value (self, _value):
        self._value = _value
        
    def get_left (self):
        return self._left

    def set_left (self, new_left):
        self._left = new_left
        
    def get_right (self):
        return self._right
    
    def set_right (self, new_right):
        self._right = new_right


class BinarySearchTree:
         
    def __init__(self):
        self._root = None        

#### Question 03 &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;(10 Points)

Write a class `Q3Vertex` as a subclass of `TreeVertex` whose constructor accepts a word as a mixed case string containing only the letters of the alphabet such as `'Hidden'` or `'valley'` and stores this as a protected instance variable. Add an `insert_word` function that positions an inserted word in a leaf vertex sorted alphabetically ignoring case.
```python
class Q3Vertex(TreeVertex):

    def __init__ (self, word = ""):
        self._value = word

    def insert_word (self, word):
        '''
        Insert a word as a leaf vertex in either left or right subtree with in-order position
        determined alphabetically ignoring case.
        '''
```
For example:
```python
vertex = Q3Vertex("Hidden")
vertex.insert_word("a")
vertex.insert_word("valley")
```
would result in a `Q3Vertex` with "Hidden" having left lead node "a" and right leaf node "valley". The in-order description of the tree would be "a", "Hidden", "valley".

In [2]:
class TreeVertex:
    def __init__(self, value=None):
        self._value = value
        self._left = None
        self._right = None
        self._parent = None  # Optional: used for easier upward traversal

    def get_value(self):
        return self._value

    def set_value(self, value):
        self._value = value

    def get_left(self):
        return self._left

    def set_left(self, new_left):
        self._left = new_left
        if new_left:
            new_left._parent = self

    def get_right(self):
        return self._right

    def set_right(self, new_right):
        self._right = new_right
        if new_right:
            new_right._parent = self

    def get_parent(self):
        return self._parent

    def set_parent(self, parent):
        self._parent = parent

    def num_children(self):
        count = 0
        if self._left:
            count += 1
        if self._right:
            count += 1
        return count


    def only_child(self):
        return self._left if self._left else self._right

    def min_value(self):
        current = self
        while current.get_left():
            current = current.get_left()
        return current.get_value()

    def __str__(self):
        return self._value


In [3]:
class Q3Vertex(TreeVertex):
    def __init__(self, word=""):
        # Call superclass constructor to initialize TreeVertex
        super().__init__(word)

    def insert_word(self, word):
        # Start from the current node (root)
        current = self
        new_node = Q3Vertex(word)

        while True:
            # Compare using lowercase (case-insensitive)
            if word.lower() < current.get_value().lower():
                # Go left if word is less
                if current.get_left() is None:
                    current.set_left(new_node)
                    break
                current = current.get_left()
            else:
                # Go right if word is greater or equal
                if current.get_right() is None:
                    current.set_right(new_node)
                    break
                current = current.get_right()


In [4]:
# Sample test case

vertex = Q3Vertex("Hidden")
vertex.insert_word("a")
vertex.insert_word("valley")
assert vertex.get_left().get_value() == 'a'
assert vertex.get_right().get_value() == 'valley'

In [ ]:
# Testing Cell (Do NOT modify this cell)

#### Question 04 &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;(15 Points)

Extend `Q3Vertex` in a class `Q4Vertex` and add the function:
```python
    def the_last_word (self):
```
that returns the last word of an in-order traversal of vertices from the root of any tree or subtree. For example, in the in-order example used above "a", "Hidden", "valley", `vertex.the_last_word()` would return `"valley"`.

In [5]:
class Q4Vertex(Q3Vertex):
    def the_last_word(self):
        '''
        Returns the last word in an in-order traversal,
        which is the rightmost node in the tree.
        '''
        current = self
        while current.get_right():
            current = current.get_right()
        return current.get_value()


In [6]:
# Sample test case

vv = Q4Vertex("Hidden")
vv.insert_word("a")
vv.insert_word("valley")
assert vv.the_last_word() == 'valley'

In [ ]:
# Testing Cell (Do NOT modify this cell)

#### Question 5 &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;(15 Points)

Extend `Q4Vertex` in a class `Q5Vertex` that adds a function:
```python
    def in_order_upper_case (self):
```
that returns the in-order traversal of the tree and returns the words as a string which concatenates each word separated by a space into a single string.

For example:
```python
    vertex.in_order_upper_case()
```
would return a string: `'A HIDDEN VALLEY'`.

In [7]:
class Q5Vertex(Q4Vertex):
    def in_order_upper_case(self):
        '''
        Returns a string of all words in in-order traversal,
        converted to uppercase and separated by spaces.
        '''
        words = []

        def in_order(node):
            if node:
                in_order(node.get_left())                  # Traverse left subtree
                words.append(node.get_value().upper())     # Add current word in UPPERCASE
                in_order(node.get_right())                 # Traverse right subtree

        in_order(self)  # Start from root (self)
        return " ".join(words)  # Concatenate with space


In [8]:
# Sample test case

vv = Q5Vertex("Hidden")
vv.insert_word("a")
vv.insert_word("valley")
assert vv.in_order_upper_case() == 'A HIDDEN VALLEY'

In [ ]:
# Testing Cell (Do NOT modify this cell)

#### Question 6 &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;(15 Points)

Create a class Q6BinarySearchTree that implements a binary search tree for words in a sentence and sorts them alphabetically ignoring case. The constructor should take a sentence such as:
`"Loveliest of trees the cherry now Is hung with bloom along the bough"` and separate individual words before adding them to the contained BST.

```python
class Q6BinarySearchTree(): 
    def __init__(self, sentence):
```   
The tree should have a method:
```python
    def contains(self, word):
```   
that searches the BST for a word and returns `True` when the word appears in the sentence **ignoring case**. For example, given the sentence above:
```python
    tree = Q6BinarySearchTree("Loveliest of trees the cherry now ...")
    tree.contains("the") == True
    tree.contains("THE") == True # Because case ignored
    tree.contains("python") == False
```

In [9]:
class Q6BinarySearchTree:
    def __init__(self, sentence):
        # Split sentence into words and ignore empty strings
        words = sentence.split()

        # Initialize root with the first word using Q5Vertex
        if words:
            self.root = Q5Vertex(words[0])
            for word in words[1:]:
                self.root.insert_word(word)
        else:
            self.root = None  # Handle empty sentence

    def contains(self, word):
        '''
        Returns True if the word is in the tree, ignoring case.
        '''
        current = self.root
        word_lower = word.lower()

        while current:
            current_word = current.get_value().lower()

            if word_lower == current_word:
                return True
            elif word_lower < current_word:
                current = current.get_left()
            else:
                current = current.get_right()

        return False


In [10]:
# Sample test cases

tree = Q6BinarySearchTree("Loveliest of trees the cherry now ...")
assert tree.contains("the")
assert tree.contains("THE") # Because case ignored
assert not tree.contains("python")

In [ ]:
# Testing Cell (Do NOT modify this cell)

#### Question 7 &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;(15 Points)

Extend `Q6BinarySearchTree` in a class `Q7BinarySearchTree` that is capable of storing at each vertex of the BST both the word and the number of times it occurs in the sentence passed to the constructor. To do this, you will need to create a new vertex class that extends a tree vertex class from a previous question. Words are added to the tree alphabetically in **lower case**. 

Your class must include a method `frequency` that traverses the tree in order and returns a list of tuples where each tuple consists of the (word, occurrences) - where occurrences is the number of times the word occurs in the sentence.
```python
class Q7BinarySearchTree(Q6BinarySearchTree):
    def __init__(self, sentence):
        ''' 
        Accepts an input sentence, converts it to lowercase and splits into
        words which are stored in the BST alphabetically ignoring case.
        '''
        
    def frequency(self):    
        '''
        Returns a list of tuples containing the in-order traversal of the  
        tree and the number of times the word appears in the sentence.
        Returned values are in lower case.
        '''
```
For example, given a sentence `"a A tree repeats"` the function `frequency` would return a list: </br>
`[('a', 2), ('repeats', 1), ('tree', 1)]`</br>
because the sentence contains two instances of 'a' when converted to lower case.


In [11]:
class Q7Vertex(TreeVertex):
    def __init__(self, word):
        super().__init__(word.lower())  # Store word in lowercase
        self._count = 1                 # First insertion = count 1

    def insert_word(self, word):
        word = word.lower()
        current = self
        while True:
            if word == current.get_value():
                current._count += 1     # Increment if duplicate
                break
            elif word < current.get_value():
                if current.get_left() is None:
                    current.set_left(Q7Vertex(word))
                    break
                current = current.get_left()
            else:
                if current.get_right() is None:
                    current.set_right(Q7Vertex(word))
                    break
                current = current.get_right()

    def get_count(self):
        return self._count
class Q7BinarySearchTree(Q6BinarySearchTree):
    def __init__(self, sentence):
        words = sentence.lower().split()
        if words:
            self.root = Q7Vertex(words[0])
            for word in words[1:]:
                self.root.insert_word(word)
        else:
            self.root = None

    def frequency(self):
        result = []

        def in_order(node):
            if node:
                in_order(node.get_left())
                result.append((node.get_value(), node.get_count()))
                in_order(node.get_right())

        in_order(self.root)
        return result


In [12]:
# Sample test case

q7 = Q7BinarySearchTree("a A tree repeats")
# This list is ordered by in-order traversal
assert q7.frequency() == [('a', 2), ('repeats', 1), ('tree', 1)]



## 🧪 Q5. Student ID Binary Search Tree (40 Points)

You are given an **unsorted list of student IDs** such as:

```python
['s2023001', 's2023099', 's2023056', 's2023009', 's2023012', 's2023080', 's2023077']
```

Your task is to implement a **Binary Search Tree (BST)** that stores these student IDs in **sorted order**. You are required to:

---

### 🔧 Instructions:

- Use the provided `TreeVertex` class to represent each node in the tree.
- Create a class called `StudentIDSearchTree` that implements the following methods:

---

### 🧩 Required Methods:

| Method | Description |
|--------|-------------|
| `insert(student_id)` | Inserts a student ID into the tree |
| `delete(student_id)` | Deletes a student ID from the tree |
| `find(student_id)` | Returns `True` if the ID exists, otherwise `False` |
| `all_sorted_ids()` | Returns all student IDs in sorted (in-order) order |
| `print_values()` | Prints the student IDs in sorted order or `"empty root"` if tree is empty |
| `get_height()` | Returns the height (maximum depth) of the tree |
| `count_nodes()` | Returns the total number of nodes in the tree |
| `is_valid_bst()` | Returns `True` if the tree structure satisfies BST rules |
| `trim_tree(low, high)` | Removes nodes not in the specified inclusive range of IDs |

> **Note:** You **must** implement these using BST logic — **do not use list sorting functions**.

---

### ✅ Example Usage:

After inserting all student IDs, a call to:

```python
tree.all_sorted_ids()
```

Should return:

```python
['s2023001', 's2023009', 's2023012', 's2023056', 's2023077', 's2023080', 's2023099']
```

---

### 📌 Requirements:

- Apply **Object-Oriented Programming (OOP)** principles for both `TreeVertex` and `StudentIDSearchTree` classes.
- Provide at least **one test case** for each method.
- Ensure all **string comparisons** follow **lexicographical order** (default Python `<`, `>`, `==` behavior for strings).


In [13]:
class TreeVertex:
    def __init__(self, value=None):
        self._value = value
        self._left = None
        self._right = None
        self._parent = None  # Optional: used for easier upward traversal

    def get_value(self):
        return self._value

    def set_value(self, value):
        self._value = value

    def get_left(self):
        return self._left

    def set_left(self, new_left):
        self._left = new_left
        if new_left:
            new_left._parent = self

    def get_right(self):
        return self._right

    def set_right(self, new_right):
        self._right = new_right
        if new_right:
            new_right._parent = self

    def get_parent(self):
        return self._parent

    def set_parent(self, parent):
        self._parent = parent

    def num_children(self):
        count = 0
        if self._left:
            count += 1
        if self._right:
            count += 1
        return count


    def only_child(self):
        return self._left if self._left else self._right

    def min_value(self):
        current = self
        while current.get_left():
            current = current.get_left()
        return current.get_value()

    def __str__(self):
        return self._value


In [14]:
class StudentIDSearchTree:
    def __init__(self):
        # Initialize the BST with an empty root node
        self.root = None

    def insert(self, student_id):
        # Create a new TreeVertex with the given student ID
        new_node = TreeVertex(student_id)

        # If the tree is empty, assign root to the new node
        if not self.root:
            self.root = new_node
            return

        # Start traversal from the root
        current = self.root
        while True:
            # If the new ID is less than current, go left
            if student_id < current.get_value():
                # If left child doesn't exist, insert here
                if not current.get_left():
                    current.set_left(new_node)
                    break
                # Move to the left child
                current = current.get_left()
            # If the new ID is greater than current, go right
            elif student_id > current.get_value():
                # If right child doesn't exist, insert here
                if not current.get_right():
                    current.set_right(new_node)
                    break
                # Move to the right child
                current = current.get_right()
            else:
                # Duplicate value found, do not insert
                break

    def find(self, student_id):
        # Start searching from the root
        current = self.root
        while current:
            # Return True if match found
            if student_id == current.get_value():
                return True
            # Go left if target is smaller
            elif student_id < current.get_value():
                current = current.get_left()
            # Go right if target is larger
            else:
                current = current.get_right()
        # If traversal ends, ID not found
        return False

    def search_vertex(self, value):
        # Helper method: returns (found, node, parent)
        parent = None
        current = self.root
        while current:
            # Return node and parent if match is found
            if current.get_value() == value:
                return True, current, parent
            # Update parent and go left
            elif value < current.get_value():
                parent = current
                current = current.get_left()
            # Update parent and go right
            else:
                parent = current
                current = current.get_right()
        # Not found case
        return False, None, None

    def delete(self, value):
        # Search for the node and its parent
        found, vertex, parent = self.search_vertex(value)

        # If value doesn't exist, return False
        if not found:
            return False

        # Count the number of children the node has
        children = vertex.num_children()

        if children == 0:
            # Case 1: No children (leaf node)
            if parent is None:
                self.root = None  # Deleting root
            elif parent.get_left() == vertex:
                parent.set_left(None)  # Remove from left of parent
            else:
                parent.set_right(None)  # Remove from right of parent

        elif children == 1:
            # Case 2: One child
            child = vertex.only_child()  # Get the only child
            if parent is None:
                self.root = child  # Replace root with child
                child.set_parent(None)
            elif parent.get_left() == vertex:
                parent.set_left(child)  # Attach child to left of parent
            else:
                parent.set_right(child)  # Attach child to right of parent

        else:
            # Case 3: Two children
            # Find the in-order successor value (smallest in right subtree)
            successor_value = vertex.get_right().min_value()
            # Delete that successor node (guaranteed to be leaf or one-child)
            self.delete(successor_value)
            # Replace current node's value with successor's value
            vertex.set_value(successor_value)

        return True

    def all_sorted_ids(self):
        # Returns all student IDs in in-order traversal (sorted)
        result = []

        def in_order(node):
            if node:
                in_order(node.get_left())           # Visit left subtree
                result.append(node.get_value())     # Visit current node
                in_order(node.get_right())          # Visit right subtree

        in_order(self.root)
        return result

    def print_values(self):
        # Prints sorted IDs or 'empty root' if tree is empty
        if self.root is None:
            print("empty root")
            return

        def in_order(node):
            if node:
                in_order(node.get_left())           # Traverse left
                print(node.get_value(), end=' ')    # Print current node
                in_order(node.get_right())          # Traverse right

        in_order(self.root)
        print()  # Newline after printing

    def get_height(self):
        # Returns height of the tree (-1 for empty tree)
        def height(node):
            if node is None:
                return -1
            return 1 + max(height(node.get_left()), height(node.get_right()))

        return height(self.root)

    def count_nodes(self):
        # Returns total number of nodes in the tree
        def count(node):
            if node is None:
                return 0
            return 1 + count(node.get_left()) + count(node.get_right())

        return count(self.root)

    def is_valid_bst(self):
        # Validates whether the tree satisfies BST properties
        def validate(node, low, high):
            if node is None:
                return True
            val = node.get_value()
            if (low is not None and val <= low) or (high is not None and val >= high):
                return False
            return validate(node.get_left(), low, val) and validate(node.get_right(), val, high)

        return validate(self.root, None, None)

    def trim_tree(self, low, high):
        # Removes all values not in the range [low, high]

        def trim(node):
            if node is None:
                return None
            # If node value is less than low, discard left and trim right
            if node.get_value() < low:
                return trim(node.get_right())
            # If node value is greater than high, discard right and trim left
            elif node.get_value() > high:
                return trim(node.get_left())
            else:
                # Recursively trim both sides
                node.set_left(trim(node.get_left()))
                node.set_right(trim(node.get_right()))
                return node

        # Update the root with trimmed tree
        self.root = trim(self.root)
